<a href="https://colab.research.google.com/github/SebastianEspitiaS/SebastianEspitiaS/blob/main/Prueba1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.auth import default

import gspread
from gspread_dataframe import set_with_dataframe

auth.authenticate_user()
creds, _ = default()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
gc = gspread.authorize(creds)
sheet_file = gc.open_by_url('https://docs.google.com/spreadsheets/d/17tJO1aQJkxxNRR81Cc42-XzQlW2ZCqGI8jCTiIt2KPI/edit?gid=0#gid=0')

In [ ]:
sheet_empleado = sheet_file.get_worksheet_by_id(0)
rows = sheet_empleado.get_all_values(date_time_render_option = 'FORMATTED_STRING', value_render_option='FORMULA')

In [ ]:
empleado_df = pd.DataFrame(rows[1:], columns=rows[0])

In [ ]:
empleado_df['Fecha Inicio Proyecto'] = pd.to_datetime(empleado_df['Fecha Inicio Proyecto'],  dayfirst=True)
empleado_df['Fecha Fin Proyecto'] = pd.to_datetime(empleado_df['Fecha Fin Proyecto'],  dayfirst=True)

In [ ]:
# @title
# !pip install faker

# import random
# import pandas as pd
# from faker import Faker
# fake = Faker()

# def agregar_nombres_apellidos(df):
#     # Generar nombres y apellidos aleatorios
#     df['Nombre'] = [fake.first_name() for _ in range(len(df))]
#     df['Apellido'] = [fake.last_name() for _ in range(len(df))]
#     return df

# empleado_df = agregar_nombres_apellidos(empleado_df)

# def agregar_nombres_empresas(df):
#     # Generar nombres de empresas aleatorios
#     df['Empresa'] = [fake.company() for _ in range(len(df))]
#     return df

# empleado_df = agregar_nombres_empresas(empleado_df)

# # Listas de Paises y ciudades
# paises_ciudades = {
#     'Estados Unidos': ['Nueva York', 'Los Ángeles', 'Chicago'],
#     'España': ['Madrid', 'Barcelona', 'Valencia'],
#     'México': ['Ciudad de México', 'Guadalajara', 'Monterrey'],
#     'Colombia': ['Bogotá', 'Medellín', 'Cali'],
#     'Alemania': ['Berlín', 'Múnich', 'Fráncfort'],
# }

# def agregar_ciudades_paises(df):
#     # Seleccionar un Pais aleatorio y su ciudad correspondiente
#     paises = list(paises_ciudades.keys())
#     df['Pais'] = [random.choice(paises) for _ in range(len(df))]
#     df['Ciudad'] = [random.choice(paises_ciudades[pais]) for pais in df['Pais']]
#     return df

# empleado_df = agregar_ciudades_paises(empleado_df)

# empleado_df['Fecha Fin Proyecto'] = empleado_df.apply(lambda row: row['Fecha Inicio Proyecto'] + pd.Timedelta(days=random.randint(7, 365)) if row['Estado del Proyecto'] == 'Completado' else np.nan, axis=1)

In [ ]:
empleado_df.head()

,ID,Nombre,Apellido,Edad,Ciudad,Pais,Profesion,Empresa,Fecha Ingreso Empresa,AñoIngreso,Salario,Proyecto,Estado del Proyecto,Fecha Inicio Proyecto,Fecha Fin Proyecto,Comentarios
0,1,Lori,Goodman,35,Madrid,España,Técnico/a,Greer-Brandt,10/04/2021 0:00:00,2021,3500,Soporte técnico,En proceso,2022-01-10,NaT,Pendiente de aprobación del cliente
1,2,Dalton,Pierce,28,Madrid,España,Analista,Morris Inc,7/09/2019 0:00:00,2019,2800,Elaboración de reportes,Terminado,2021-03-22,2021-04-26,Bajo presupuesto asignado
2,3,Mark,Perez,42,Medellín,Colombia,Marketing,Smith and Sons,19/10/2016 0:00:00,2016,4100,Posicionamiento de marca,Terminado,2019-07-15,2019-10-04,Proyecto finalizado con éxito
3,4,Michael,Howell,30,Berlín,Alemania,Marketing Digital,Morris Inc,24/03/2020 0:00:00,2020,2900,Publicidad en redes sociales,Terminado,2020-11-01,2021-06-03,Bajo presupuesto asignado
4,5,Alexander,Clark,25,Chicago,Estados Unidos,Gerente de Proyectos,Morris Inc,11/12/2019 0:00:00,2019,2700,Gestión de proyectos,Terminado,2022-05-14,2023-02-02,Bajo presupuesto asignado


In [ ]:
empleado_df['Año Ingreso'] = pd.to_datetime(empleado_df['Fecha Ingreso Empresa'], dayfirst=True).dt.year
analisis_edad_df = empleado_df.groupby(['Año Ingreso', 'Empresa', 'Edad'])['Edad'].count().reset_index(name='Cantidad de Empleados por edad')

In [ ]:
analisis_edad_df = empleado_df.groupby(['Año Ingreso', 'Empresa'])['Edad'].mean().reset_index(name='EdadPromedio')
analisis_edad_df['EdadPromedio']  = np.ceil(analisis_edad_df['EdadPromedio']).astype(int)

In [ ]:
analisis_edad_df.head()

,Año Ingreso,Empresa,EdadPromedio
0,2014,Greer-Brandt,45
1,2014,Morris Inc,45
2,2014,Smith and Sons,38
3,2015,Greer-Brandt,39
4,2015,Morris Inc,41


In [ ]:
analisis_edad_df_copy = analisis_edad_df.copy()

**Historigrama de la media de edad**


---
Media de edad de los empleados por empresa a lo largo del tiempo


In [ ]:
# @title
import altair as alt

# Create a line chart of the average age over time for each company
chart = alt.Chart(analisis_edad_df).mark_line(point=True).encode(
    x=alt.X('Año Ingreso:O', title='Año de Ingreso'),
    y=alt.Y('EdadPromedio:Q', title='Edad Promedio', scale=alt.Scale(domain=[20, 46])),
    color=alt.Color('Empresa:N', title='Empresa', legend=alt.Legend(titleFontSize=16, labelFontSize=14)),
).properties(
    width=1000,  # Adjust width as needed for a larger chart
    height=600,  # Adjust height as needed for a larger chart
    title='Edad promedio de los Empleados por Empresa a lo largo del Tiempo'
).configure_legend(
    labelFontSize=14,  # Increase label font size for better readability
    titleFontSize=16
)

chart

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [ ]:
sheet_resultado =  sheet_file.get_worksheet_by_id(1195046912)
set_with_dataframe(sheet_resultado, analisis_edad_df_copy, resize=True)
sheet_resultado.set_basic_filter()

{'spreadsheetId': '17tJO1aQJkxxNRR81Cc42-XzQlW2ZCqGI8jCTiIt2KPI',
 'replies': [{}]}

In [ ]:
empleado_ciudad_pais = empleado_df.groupby(['Pais', 'Ciudad', 'Empresa'])['ID'].count().reset_index(name='Talento por pais/ciudad')

In [ ]:
empleado_ciudad_pais.head()

,Pais,Ciudad,Empresa,Talento por pais/ciudad
0,Alemania,Berlín,Greer-Brandt,3
1,Alemania,Berlín,Morris Inc,6
2,Alemania,Berlín,Smith and Sons,3
3,Alemania,Fráncfort,Greer-Brandt,4
4,Alemania,Fráncfort,Morris Inc,3


**Cantidad Empleados Por Pais**


---


Cantidad de empleados por pais de cada empresa

In [ ]:
# @title
import altair as alt

# Agrupa los datos por país y empresa y suma la cantidad de empleados
empleados_por_pais_empresa = empleado_ciudad_pais.groupby(['Pais', 'Empresa'])['Talento por pais/ciudad'].sum().reset_index()

# Crea un gráfico de barras con la cantidad de empleados por país y empresa
chart = alt.Chart(empleados_por_pais_empresa).mark_bar().encode(
    x=alt.X('Empresa:N', axis=alt.Axis(labelFontSize=14)),
    y=alt.Y('sum(Talento por pais/ciudad):Q', scale=alt.Scale(domain=(0, 25)), axis=alt.Axis(labelFontSize=14)),
    color=alt.Color('Empresa:N', scale=alt.Scale(scheme='category20')),
    column=alt.Column('Pais:N', header=alt.Header(labelFontSize=16))
).properties(
    title='Cantidad de empleados por país y empresa',
    width=200,
    height=400
)

chart

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [ ]:
sheet_empleados_por_pais_empresa = sheet_file.get_worksheet_by_id('1161899332')
set_with_dataframe(sheet_empleados_por_pais_empresa, empleados_por_pais_empresa, resize=True)
sheet_empleados_por_pais_empresa.set_basic_filter()

{'spreadsheetId': '17tJO1aQJkxxNRR81Cc42-XzQlW2ZCqGI8jCTiIt2KPI',
 'replies': [{}]}

In [ ]:
salario_profesion_pais = empleado_df.groupby(['Pais', 'Empresa', 'Profesion'])['Salario'].mean().reset_index(name='Salario Promedio')

In [ ]:
salario_profesion_pais

,Pais,Empresa,Profesion,Salario Promedio
0,Alemania,Greer-Brandt,Abogado/a,3500.0
1,Alemania,Greer-Brandt,Analista,3000.0
2,Alemania,Greer-Brandt,Científico/a,3400.0
3,Alemania,Greer-Brandt,Director/aAnalista de Sistemas,3350.0
4,Alemania,Greer-Brandt,Diseñador/a Gráfico/a,3600.0
...,...,...,...,...
138,México,Smith and Sons,Economista,4700.0
139,México,Smith and Sons,Gerente,3380.0
140,México,Smith and Sons,Marketing,3850.0
141,México,Smith and Sons,Marketing Digital,2900.0


In [ ]:
import altair as alt

# Create a bar chart with stacked bars
chart = alt.Chart(salario_profesion_pais).mark_bar().encode(
    x=alt.X('Profesion:N', sort='-y'),  # Sort professions by descending average salary
    y=alt.Y('Salario Promedio:Q', aggregate='mean'),  # Calculate mean salary for each profession
    color=alt.Color('Empresa:N'),  # Use color to represent different companies
    tooltip=['Pais', 'Profesion', 'Empresa', 'Salario Promedio'] # Add tooltip to display details on hover
).properties(
    title='Salario Promedio por Profesion, Empresa y Pais'
).facet(column='Pais:N')

# Display the chart
chart

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.FacetChart(...)